In [ ]:
import dask
import dask.array as da
from dask_ml.model_selection import KFold
import xgboost as xgb
from dask.distributed import LocalCluster

X = da.asarray(X)
y = da.asarray(y)

X = X.rechunk((1000, X.shape[1]))
y = y.rechunk((1000,))

params = {
    'objective': 'multi:softmax',
    'max_depth': 4,
    'eta': 0.01,
    'subsample': 0.5,
    'min_child_weight': 0.5,
    'num_class': 7
}

n_splits = 5
cv = KFold(n_splits=n_splits, shuffle=True, random_state=94)

predictions = {}

with LocalCluster(n_workers=8) as cluster:
    with cluster.get_client() as client:
        for i, (train, test) in enumerate(cv.split(X, y)):
            X_train = X[train, :]
            X_test = X[test, :]
            y_train = y[train]
            y_test = y[test]

            d_train = xgb.dask.DaskDMatrix(client, X_train, y_train, enable_categorical=True)
            model = xgb.dask.train(client, params=params, dtrain=d_train)
            predictions[f'fold_{i}'] = xgb.dask.predict(client, model, X_test)

C:\Users\local\venv\lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 64386 instead
  warnings.warn(
C:\Users\local\AppData\Local\Programs\Python\Python310\Lib\contextlib.py:142: UserWarning: Creating scratch directories is taking a surprisingly long time. (2.74s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
C:\Users\local\venv\lib\site-packages\distributed\client.py:1606: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| numpy   | 2.1.1  | 2.1.1     | 2.0.2   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))
C:\Users\local\venv\lib\site-packages\dask\a

In [ ]:
import dask
import dask.array as da
from dask_ml.model_selection import KFold
import xgboost as xgb
from dask.distributed import LocalCluster
import joblib
from sklearn.metrics import confusion_matrix, classification_report

X = da.asarray(X).rechunk((1000, X.shape[1]))
y = da.asarray(y).rechunk((1000,))

params = {
    'objective': 'multi:softmax',
    'max_depth': 4,
    'eta': 0.01,
    'subsample': 0.5,
    'min_child_weight': 0.5,
    'num_class': 7
}

n_splits = 5
cv = KFold(n_splits=n_splits, shuffle=True, random_state=94)

with LocalCluster(n_workers=8) as cluster:
    with cluster.get_client() as client:
        for i, (train, test) in enumerate(cv.split(X, y)):
            X_train = X[train, :]
            X_test = X[test, :]
            y_train = y[train]
            y_test = y[test]

            d_train = xgb.dask.DaskDMatrix(client, X_train, y_train, enable_categorical=True)
            model = xgb.dask.train(client, params=params, dtrain=d_train)

            joblib.dump(model, f'model_fold_{i}.pkl')

            predictions_fold = xgb.dask.predict(client, model, X_test)
            predictions_fold = predictions_fold.compute()

            cm = confusion_matrix(y_test.compute(), predictions_fold)
            print(f"Macierz klasyfikacji dla fold {i}:\n", cm)
            print(classification_report(y_test.compute(), predictions_fold))

In [ ]:
import dask.array as da
from dask_ml.model_selection import GridSearchCV
import xgboost as xgb
from dask.distributed import Client

X = da.asarray(X).rechunk((1000, X.shape[1]))
y = da.asarray(y).rechunk((1000,))

client = Client()

param_grid = {
    'max_depth': [3, 4, 5],
    'eta': [0.01, 0.1, 0.2],
    'subsample': [0.5, 0.75, 1.0],
    'min_child_weight': [1, 2, 3],
    'num_class': [7]
}

model = xgb.dask.DaskXGBClassifier(objective='multi:softmax', random_state=94)

grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X, y)

best_params = grid_search.best_params_
print("Najlepsze parametry:", best_params)

initial_params = {
    'max_depth': 4,
    'eta': 0.01,
    'subsample': 0.5,
    'min_child_weight': 0.5,
    'num_class': 7
}

print("Początkowe parametry:", initial_params)